In [2]:
from Bond import BondMultiPeriod
from Stock import StockMultiPeriod

# Introduction <a class="anchor" id="introduction"></a>

## Table of content
* [Introduction](#introduction)
    * [Goal](#goal)
    * [Model setup](#model_setup)
        * [Bond](#bond)
        * [Derivative](#derivative)
        * [Stock](#stock)
* [Pricing](#pricing)
* [Arbitrage and completeness](#arbitrage_completeness)
    * [Arbitrage](#arbitrage)
    * [Completeness](#completeness)

***
## Goal <a class="anchor" id="goal"></a>

The goal of this notebook is to generalize the one period model and its properties.

We will:

1. Define the model setup (tree)
    - Bond
    - Stock
 
2. Arbitrage and completeness
3. Pricing

***
## Model setup <a class="anchor" id="model_setup"></a>

As in the one period model, we consider a bond and a stock. 

### Bond <a class="anchor" id="bond"></a>

In [3]:
# Define multi period bond
bond = BondMultiPeriod(r = 0.1, T = 4)

# Bond values
for i in range(0, len(bond.values)):
    print("At t = %s, the bond is worth %s")%(i+1, bond.values[i])

At t = 1, the bond is worth 1.0
At t = 2, the bond is worth 1.1
At t = 3, the bond is worth 1.21
At t = 4, the bond is worth 1.33


### Derivative <a class="anchor" id="derivative"></a>

<b>Definition: (derivative)</b> A financial derivative or contingent claim is a random variable of the form

$$ X = \phi(S_{T}) $$

where the contract function $\phi$ is a real valued function.

### Stock <a class="anchor" id="stock"></a>

In [4]:
# Defining a stock
stock = StockMultiPeriod(s = 80, u = 1.5, d = 0.5, p = 0.6, T = 3)
# Forward stock value process
for i in range(0, len(stock.values)):
    print("At t=%s: S-%s = %s")%(i+1, i, stock.values[i])

At t=1: S-0 = 80
At t=2: S-1 = [120.  40.]
At t=3: S-2 = [180.  60.  20.]
At t=4: S-3 = [270.  90.  30.  10.]


In [5]:
# Compute a derivative with strike price K = 80 on the stock
print("The derivative: (t=%s): %s")%(stock.T, stock.compute_derivative(k = 80))

The derivative: (t=3): [190.  10.   0.   0.]


# Pricing <a class="anchor" id="pricing"></a>

If $X = \phi(S_{T})$ is replicated by the portfolio h then:

$$
\Pi(t;X) = V_{h}^{t} \text{t=0, ..., T}
$$

The replicating portfolio is given by

$$\begin{equation} 
\begin{cases}
  x_{t}(k) = \frac{1}{1+R} \frac{u V_{t}(k) - d V_{t}(k+1)}{u-d} \\
  y_{t}(k) = \frac{1}{S_{t-1}} \frac{V_{t}(k+1) - V_{t}(k)}{u-d}
\end{cases}
\end{equation}$$

From the above algorithm and how the portfolio value is computed we can obtain a risk neutral valuation formula.

In [6]:
# Price the options by computing risk neutral validations equations backward
stock.price_option(k = 80, r = 0)
for i in range(0, len(stock.prices)):
    print("t=%s: Pi-%s = %s")%(i+1, i, stock.prices[i])

t=1: Pi-0 = [27.5]
t=2: Pi-1 = [52.5  2.5]
t=3: Pi-2 = [100.   5.   0.]
t=4: Pi-3 = [190.  10.   0.   0.]


<b>Proposition</b> The arbitrage free price at t = 0 of the claim X is given by

$$
\Pi(0;X)=V_{0}=\frac{1}{(1+R)^{T}}E^{Q}[X]
$$

where Q denotes the martingale measure. More explicitely, this formula is given by:

$$
\Pi(0;X)=V_{0}=\frac{1}{(1+R)^{T}}\sum_{k=0}^{T}\binom{T}{k}q^{k}(1-q)^{T-k}\phi(s u^{k} d^{T-k})
$$

# Arbitrage and completeness <a class="anchor" id="arbitrage_completeness"></a>

## Arbitrage <a class="anchor" id="arbitrage"></a>

<b>Definition: (Portfolio)</b> A portfolio strategy is a stochastic process 

$$ \{ h_{t} = (x_{t}, y_{t}); t = 1, ..., T \} $$

such that $h_{t}\in \sigma\{ S_{0}, ..., S_{t-1} \}$.

By convention: $h_{0} = h_{1}$. We have that:

- $x_{t}$ number of SEK in the bank $(t-1, t]$
- $y_{t}$ number of stocks you own $(t-1, t]$

<b>Definition: (Value process)</b> Value process $V^{h}$ corresponding to the portfolio strategy $h$ is given by  $$ V_{t}^{h}=x_{t}(1+R) + y_{t}S_{t} $$

<b>Definition: (Self financing)</b> Self financing if $$ x_{t}(1+R) = y_{t}S_{t} = x_{t+1} + y_{t+1}S_{t} $$

<b>Definition: (Arbitrage portfolio)</b> Arbitrage portfolio if $h$ is self-financing and

- $V^{h}(0)=0$
- $P(V^{h}_{T}(1)\geq 0) = 1$

- $P(V^{h}_{T}(1) > 0) > 0$

In [8]:
# Compute the replicating portfolio forward (?)
x, y = stock.compute_portfolio(0)

for i in range(0, len(x)):
    print("At t=%s: x = %s")%(i+1, x[i])
    
for i in range(0, len(y)):
    print("At t=%s: y = %s")%(i+1, y[i])

At t=1: x = [-22.5]
At t=2: x = [-42.5  -2.5]
At t=3: x = [-80.  -5.   0.]
At t=1: y = [0.625]
At t=2: y = [0.79166667 0.125     ]
At t=3: y = [1.         0.16666667 0.        ]


***
## Completeness <a class="anchor" id="completeness"></a>

The claim $X = \phi(S_{T})$ is reachable if there exists a self-financing portfolio strategy such that  $$ V_{T}^{h} = \phi(S_{T}) \text{ with probability 1} $$

If such a portfolio $h$ exists is said to be a hedging or replicating portfolio for the claim $X$.

<b>Proposition</b> The model is free of arbitrage if and only if: $d < 1+R < u$

<b>Definition</b> Q is a martingale measure if 
- $0 < q < 1 \text{ (means) } Q \sim P$
- $s=\frac{1}{1+R}E^{Q}[S_{t+1}\mid S_{t}=s]$

<b>Proposition</b> The martingale probabilities are given by $$ (q = \frac{1+R - d}{u-d}, 1-q) $$

<b>Central Proposition</b> The multiperiod binomial model is complete (i.e. all claims are reachable)

# Questions 

- Why do we need to define a bond to say id a model is free of arbitrage?